# Library

In [12]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import re
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [13]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [14]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [15]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [16]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

## 페이지 접속

In [17]:
# 페이지 주소
url = 'https://shopping.naver.com/ns/home'

### requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [18]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

### selenium 작동

In [19]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

### 검색 창에 검색어 입력

In [20]:
# 검색창 정보
search_window_xpath = '//*[@id="gnb-gnb"]/div[2]/div/div[2]/div[1]/form/div/div/div/div/input' # 검색 입력창
search_window = driver.find_element(By.XPATH, search_window_xpath)

# 검색어
search_keyword = '아이라인'

# 검색창에 검색어 입력
search_window.send_keys(search_keyword)

# 검색 창 요소가 존재할 때까지 기다림
search_window = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, search_window_xpath))
)

# 검색 창에 엔터 입력
search_window.send_keys(Keys.ENTER)
driver.implicitly_wait(10)

In [21]:
# 스크롤해야 정보가 로드되는 페이지
# 페이지 스크롤 다운
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 문서 높이 만큼 스크롤

# # 무한 페이지 스크롤다운
# last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

# while True:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 문서 높이 만큼 스크롤
#     # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") # # 문서 높이 만큼 스크롤(위 코드와 동일한 기능)
#     time.sleep(random.uniform(1, 3))
#     driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료

#     new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
#     if new_page_height == last_page_height:
#         break
#     else:
#         last_page_height = new_page_height

## 🔧 입력 키워드 정보 수집
- 페이지 로드되어도 보고있는 화면 근처 정보만 수집되는걸로 보임
    1. 페이지 스크롤 -> 수집 -> 페이지 스크롤 반복
    2. 수집 후 중복 제거

In [22]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'아이라인 : 네이버+ 스토어 검색'

In [23]:
# 상품 섹션
products_section = soup.select_one('div#composite-card-list')
len(products_section)

2

In [24]:
# 상품 리스트
product_list = products_section.select('li.compositeCardContainer_composite_card_container__jr8cb.composite_card_container div.basicProductCard_basic_product_card__TdrHT.basicProductCard_view_type_grid2__vKr1n')
len(product_list)

47

### 상세 페이지 링크

In [25]:
# 상품 상세 페이지
order_number = 0
product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']

'https://adcr.naver.com/adcr?x=Vp01qPh0Gu9BG5YADhK/CP///w==kQS7mMPzadzbEKXYHvLxBUBCE2JmcSQRNmuYO20uacdc+4FWKKQnzAtEQLRZFcofURnYX4To0twKZDSScCHI5oscO6yc8ncyDd1DSZgXJ7B9bbfFub3vYEsv+Qc44SQgmadlmZu7qOZViPuaghcADUAU+GL8FCOaLEZuM7hhR5ssb2kRaB3ia2l+WANbZj3MS0PFnl8WN1g1pr/YeeYb/lprfwt59zrFYM0WKyOVWeqRsqI+0GlccJP6mwXGZ+Yhxq2wiu+zh26VRDvKifhT5eLx9NTigNBRgNTzwqZbBvdMgHKbumifAW8waMk0H7PtttqwBYqaB3tIAGiseH2JPpIf6uxJEfZLzxgDryE67qn6ppixa3EgVznQQghZp1QvfLLZpmfRhCnk7kq+ZdyoyZsmZUIRGZ7l62g6qMHua/v8o69ggA2D01pNutLJM1IinUmMjFm/z4qtjOf2a0UgyMj1/atkkjrn/RYckyYIIPoxZxh9Mp8XrkJyddm+hJa0LBtzUZbdf9aC89U5L/oWAEMrnRbKLQbnJKfg1lsfvqU5VzI584iVVq0f+88i7vOmyeVx2t3UVXDr5VIA57wZVp+3FXmlSstsYLNJ+racXIJSseBH1Q6xV/gZAIQJlRzmdJfdWvjR+k2+4iVWsJJGlA0SFvgpN5x6OOVXKf1EDAm2u9MkBVu680NmFIAS2ahxgHYAPsHNQmlY+oBbV5fSdBWK1MdcMFzbTSBg7VhZANdY68565WW4puigu++kxzIaLE+UgrQaCrm9FW1zlea1OE5XQpkN26jb3qLuKlAHGlW4lhYqYjZzfbPn+Gka/1OZGKfA8G8q+DXwyzD07KR5xAAFnnBkmHk/k8odDRFpjzzemniuR/4r3hdTMIbfceqlDovOjEAHJY5I4Yf7Rt6/Zug==#channelProductNo=72

### 🔧 이미지 링크
- 이미지 저장 기능 추가

In [26]:
# 상품 썸네일 이미지 링크
order_number = 0
product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']

'https://shopping-phinf.pstatic.net/main_8482107/84821073363.5.jpg?type=f300'

### 상품 정보 섹션

In [27]:
# 상품 정보 섹션
order_number = 0
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
product_info_section

<div class="basicProductCardInformation_basic_product_card_information__7v_uc basicProductCardInformation_view_type_grid2__mlh6E" id="basic_product_card_information_7276573041"><div class="basicProductCardInformation_mall_area__xcGLH"><div class="basicProductCardInformation_advertisement_area__HzaQ_"><span class="advertisementTag_advertisement_tag__yIvim"><a class="advertisementTag_link__NAi19 _nlog_click" data-shp-area="prod.adinfo" data-shp-area-id="adinfo" data-shp-area-type="action" data-shp-inventory="prod" data-shp-nsc="svc.shopping.plusall" data-shp-page-key="100407255" data-shp-sti="shopping" href="#" role="button" target="_blank"><svg class="advertisementTag_icon__DdazQ advertisementTag_information__jKJtn" fill="none" height="18" viewbox="0 0 40 18" width="40" xmlns="http://www.w3.org/2000/svg"><rect height="17" rx="8.5" stroke="#D9D9DC" width="39" x="0.5" y="0.5"></rect><path d="M13.096 11.988c0 1.133-1.265 1.826-3.333 1.826s-3.311-.693-3.311-1.826 1.254-1.837 3.311-1.837 3.3

#### 상품 제목

In [28]:
# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

나어릴때 워터프루프 브러쉬 아이라인


#### 🔧 상품 가격
- 숫자만 추출

In [ ]:
# 상품 가격
raw_original_price = product_info_section.select_one('span.priceTag_original_price__jyZRY').text
original_price = int(re.sub('[^0-9]', '', raw_original_price))

raw_discount_rate = product_info_section.select_one('span.priceTag_discount__F_ZXz').text
discount_rate = int(re.sub('[^0-9]', '', raw_discount_rate)) / 100

raw_price = product_info_section.select_one('span.priceTag_inner_price__TctbK').text
price = int(re.sub('[^0-9]', '', raw_price))

print(raw_original_price)
print(type(original_price), original_price)
print(raw_discount_rate)
print(type(discount_rate), discount_rate)
print(raw_price)
print(type(price), price)

할인 전 판매가14,000원
<class 'int'> 14000
39%할인
<class 'float'> 0.39
8,500원
<class 'int'> 8500


#### 🔧 배송비
- 숫자만 추출

In [30]:
product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text

'배송비 3,000원'

#### 종합

In [31]:
order_number = 0

# 상세 페이지 링크
print(product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href'])


# 이미지 링크
print(product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src'])

# 상품 정보 섹션
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')

# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

# 원래 상품 가격
try:
    print(product_info_section.select_one('span.priceTag_original_price__jyZRY').text)
except:
    print('할인없음')
# 할인률
try:
    print(product_info_section.select_one('span.priceTag_discount__F_ZXz').text)
except:
    print('할인률없음')
# 현재 가격
print(product_info_section.select_one('span.priceTag_inner_price__TctbK').text)
# 배송비비
try:
    print(product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text)
except:
    print('배송비없음')

https://adcr.naver.com/adcr?x=Vp01qPh0Gu9BG5YADhK/CP///w==kQS7mMPzadzbEKXYHvLxBUBCE2JmcSQRNmuYO20uacdc+4FWKKQnzAtEQLRZFcofURnYX4To0twKZDSScCHI5oscO6yc8ncyDd1DSZgXJ7B9bbfFub3vYEsv+Qc44SQgmadlmZu7qOZViPuaghcADUAU+GL8FCOaLEZuM7hhR5ssb2kRaB3ia2l+WANbZj3MS0PFnl8WN1g1pr/YeeYb/lprfwt59zrFYM0WKyOVWeqRsqI+0GlccJP6mwXGZ+Yhxq2wiu+zh26VRDvKifhT5eLx9NTigNBRgNTzwqZbBvdMgHKbumifAW8waMk0H7PtttqwBYqaB3tIAGiseH2JPpIf6uxJEfZLzxgDryE67qn6ppixa3EgVznQQghZp1QvfLLZpmfRhCnk7kq+ZdyoyZsmZUIRGZ7l62g6qMHua/v8o69ggA2D01pNutLJM1IinUmMjFm/z4qtjOf2a0UgyMj1/atkkjrn/RYckyYIIPoxZxh9Mp8XrkJyddm+hJa0LBtzUZbdf9aC89U5L/oWAEMrnRbKLQbnJKfg1lsfvqU5VzI584iVVq0f+88i7vOmyeVx2t3UVXDr5VIA57wZVp+3FXmlSstsYLNJ+racXIJSseBH1Q6xV/gZAIQJlRzmdJfdWvjR+k2+4iVWsJJGlA0SFvgpN5x6OOVXKf1EDAm2u9MkBVu680NmFIAS2ahxgHYAPsHNQmlY+oBbV5fSdBWK1MdcMFzbTSBg7VhZANdY68565WW4puigu++kxzIaLE+UgrQaCrm9FW1zlea1OE5XQpkN26jb3qLuKlAHGlW4lhYqYjZzfbPn+Gka/1OZGKfA8G8q+DXwyzD07KR5xAAFnnBkmHk/k8odDRFpjzzemniuR/4r3hdTMIbfceqlDovOjEAHJY5I4Yf7Rt6/Zug==#channelProductNo=727

# 🔧 수집한 데이터 전처리
- 원래가격, 할인률, 현재가격 숫자만 표기하도록 정제
- 원래가격, 할인률 nan 값 처리된 부분 확인

In [ ]:
num_list = []
detail_link_list = []
img_src_list = []
title_list = []
original_price_list = []
discount_rate_list = []
price_list = []
delivery_price_list = []

for i in range(len(product_list)):
    num = i + 1

    # 상세 페이지 링크
    try:
        detail_link = product_list[i].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']
    except:
        print(i)
        # print('상세페이지링크없음')
    # 이미지 링크
    try:
        img_src = product_list[i].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']
    except:
        print(i)
        # print('이미지링크없음')
    
    # 상품 정보 섹션
    product_info_section = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
    
    ## 상품 제목
    try:
        title = product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text
    except:
        print(i)
        # print('상품제목없음')
    
    ## 원래가격
    try:
        raw_original_price = product_info_section.select_one('span.priceTag_original_price__jyZRY').text
        original_price = int(re.sub('[^0-9]', '', raw_original_price))
    except:
        original_price = np.nan
        # print('원래가격없음')
    ## 할인률
    try:
        raw_discount_rate = product_info_section.select_one('span.priceTag_discount__F_ZXz').text
        discount_rate = int(re.sub('[^0-9]', '', raw_discount_rate)) / 100
    except:
        discount_rate = np.nan
        # print('할인률없음')
    ## 현재가격
    try:
        raw_price = product_info_section.select_one('span.priceTag_inner_price__TctbK').text
        price = int(re.sub('[^0-9]', '', raw_price))
    except:
        print(i)
        # print('현재가격없음')
    ## 배송비
    try:
        delivery_price = product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text
    except:
        delivery_price = np.nan
        # print('배송비없음')

    # 정보 리스트에 담기
    num_list.append(num)
    detail_link_list.append(detail_link)
    img_src_list.append(img_src)
    title_list.append(title)
    original_price_list.append(original_price)
    discount_rate_list.append(discount_rate)
    delivery_price_list.append(delivery_price)
    price_list.append(price)


1
5
28
34
35


In [33]:
len(num_list), len(detail_link_list), len(img_src_list), len(title_list), len(original_price_list), len(discount_rate_list), len(price_list), len(delivery_price_list)

(47, 47, 47, 47, 47, 47, 47, 47)

In [34]:
data_dict = {
    '번호':num_list, 
    '상세페이지링크':detail_link_list,
    '이미지링크':img_src_list,
    '제품제목':title_list,
    '원래가격':original_price_list,
    '할인률':discount_rate_list,
    '현재가격':price_list,
    '배송비':delivery_price_list
    }
df = pd.DataFrame(data_dict)
df.head()

번호                                            상세페이지링크  \
0   1  https://adcr.naver.com/adcr?x=bxEcViLe53LQejej...   
1   2  https://adcr.naver.com/adcr?x=jrCMSOml7FTwryzc...   
2   3  https://adcr.naver.com/adcr?x=1Kq9m/2F5kFnA+sA...   
3   4  https://adcr.naver.com/adcr?x=GhXryHCzsPvsfaG6...   
4   5  https://smartstore.naver.com/main/products/551...   

                                               이미지링크  \
0  https://shopping-phinf.pstatic.net/main_848210...   
1  https://shopping-phinf.pstatic.net/main_848210...   
2  https://searchad-phinf.pstatic.net/MjAyMzA1MDh...   
3  https://shopping-phinf.pstatic.net/main_825922...   
4  https://shopping-phinf.pstatic.net/main_830583...   

                                            제품제목             원래가격    할인률  \
0                            나어릴때 워터프루프 브러쉬 아이라인  할인 전 판매가14,000원  39%할인   
1                      1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너  할인 전 판매가24,000원  31%할인   
2                      메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인  할인 전 판매가11,000원   5%할인   
3                      [공식]메이블린 라인 타투 아이 라이너 크레용  할인 전 판매가13,000원  25%할인   
4  번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임  할인 전 판매가12,900원  23%할인   

      현재가격         배송비  
0   8,500원  배송비 3,000원  
1  16,500원         NaN  
2  10,400원  배송비 3,000원  
3   9,750원         NaN  
4   9,900원  배송비 3,000원

In [35]:
df.tail()

번호                                            상세페이지링크  \
42  43  https://smartstore.naver.com/main/products/603...   
43  44  https://smartstore.naver.com/main/products/108...   
44  45  https://smartstore.naver.com/main/products/865...   
45  46  https://smartstore.naver.com/main/products/513...   
46  47  https://smartstore.naver.com/main/products/100...   

                                                이미지링크  \
42  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
43  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
44  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
45  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
46  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   

                                          제품제목             원래가격    할인률  \
42  애리조 번지지않은 안지워지는 펜 아이라이너 펄 워터프루프 아이라인펜슬1.5g              NaN    NaN   
43                        애교살 아이라인 그리기 마스카라 도구              NaN    NaN   
44                아이라인그리기 눈화장가이드 애교살만들기 마스카라가드   할인 전 판매가8,000원  60%할인   
45   워터프루프 오앤에이 아이라이너 롱라스팅 브러쉬 초슬림 아이라인 펜슬 3색상  할인 전 판매가16,000원  65%할인   
46                         비러브 마이라인 펜 아이라이너 블랙   할인 전 판매가2,900원   9%할인   

       현재가격         배송비  
42  13,000원  배송비 3,000원  
43     700원  배송비 3,000원  
44   3,200원  배송비 3,000원  
45   5,500원  배송비 3,000원  
46   2,630원  배송비 3,000원

In [36]:
df.loc[df['현재가격'].isnull()]

Empty DataFrame
Columns: [번호, 상세페이지링크, 이미지링크, 제품제목, 원래가격, 할인률, 현재가격, 배송비]
Index: []

In [37]:
df.loc[df['배송비'].isnull()]

번호                                            상세페이지링크  \
1    2  https://adcr.naver.com/adcr?x=jrCMSOml7FTwryzc...   
3    4  https://adcr.naver.com/adcr?x=GhXryHCzsPvsfaG6...   
5    6  https://smartstore.naver.com/main/products/906...   
8    9  https://smartstore.naver.com/main/products/467...   
10  11  https://smartstore.naver.com/main/products/504...   
11  12  https://smartstore.naver.com/main/products/399...   
16  17  https://smartstore.naver.com/main/products/733...   
28  29  https://smartstore.naver.com/main/products/502...   
34  35  https://smartstore.naver.com/main/products/108...   
35  36  https://smartstore.naver.com/main/products/110...   

                                                이미지링크  \
1   https://shopping-phinf.pstatic.net/main_848210...   
3   https://shopping-phinf.pstatic.net/main_825922...   
5   https://shopping-phinf.pstatic.net/main_830583...   
8   https://shopping-phinf.pstatic.net/main_822170...   
10  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
11  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
16  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
28  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
34  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   
35  data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...   

                                                제품제목             원래가격    할인률  \
1                          1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너  할인 전 판매가24,000원  31%할인   
3                          [공식]메이블린 라인 타투 아이 라이너 크레용  할인 전 판매가13,000원  25%할인   
5           잘안지워지는 점막 지워지지않는 수영장 음영 얇은 아이라인 아이라이너 펜슬  할인 전 판매가17,800원  11%할인   
8   아임유니 울트라 슬림 워터프루프 젤 아이라이너 펜슬 8컬러 안번지는 점막 아이라인 컬러  할인 전 판매가11,000원  46%할인   
10                     메이블린 뉴욕 라인 타투 아이 라이너 크레용 0.4g  할인 전 판매가13,000원  25%할인   
11             1+1 리르 룩스 젤 펜슬 아이라이너 아이라인 아이메이크업 0.5g  할인 전 판매가20,000원  50%할인   
16               헤라 이지 스타일링 아이라이너 0.5g 붓펜 아이라인 워터프루프              NaN    NaN   
28        1+1미크릿 비건 안번지는 미온수클렌징 아이라이너 0.6g(다크브라운/블랙)  할인 전 판매가24,000원  31%할인   
34            아이/그레이당일 미코 러브 앙고라 반팔 니트 가디건 3col 하트단추              NaN    NaN   
35                [아이 당일] 24fw unum 알파카 니트 자켓 3color  할인 전 판매가79,000원  24%할인   

       현재가격  배송비  
1   16,500원  NaN  
3    9,750원  NaN  
5   15,800원  NaN  
8    5,900원  NaN  
10   9,750원  NaN  
11   9,900원  NaN  
16  21,780원  NaN  
28  16,500원  NaN  
34  54,000원  NaN  
35  60,000원  NaN

In [38]:
sum(df['배송비'].isnull())

10

# 데이터 출력

In [39]:
current_date = time.strftime('%Y%m%d')
current_date

'20241219'

In [40]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241219_AM_103609'

In [41]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [42]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\20241219'

In [43]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [44]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_{search_keyword}_{current_datetime}.xlsx'), index=False)

In [11]:
driver.quit()

# END